<a href="https://colab.research.google.com/github/ItsMeDarshan/Deep-Learning/blob/master/Visual_Odometry_MML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PWCNET

####Download datasets to the virtual machine

In [ ]:
%%capture
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_scene_flow.zip

In [ ]:
%%capture
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_odometry_gray.zip

####Unzip data and store at a particular place(inside the virtual machines's provided memory)

In [ ]:

!unzip data_odometry_gray.zip -d "/content/Kitti_odo"

Archive:  data_odometry_gray.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data_odometry_gray.zip or
        data_odometry_gray.zip.zip, and cannot find data_odometry_gray.zip.ZIP, period.


In [ ]:
%%capture
!unzip data_scene_flow.zip -d "/content/Kitti2015"

In [ ]:
%%capture
#!wget http://files.is.tue.mpg.de/sintel/MPI-Sintel-complete.zip
!unzip "/content/pwcnet/content/pwcnet/MPI-Sintel-complete.zip" -d "/content/Sintel"

####Clone the PWCNET Repo(again inside the virtual machines's provided memory)

In [ ]:
%%capture
%cd /content
!rm -r pwcnet
!git clone https://github.com/daigo0927/pwcnet
%cd pwcnet
#!git checkout tf2
!git submodule init
!git submodule update

####Print current working directory and list the sub directories and files

In [ ]:
#!pip list | grep tensorflow
!echo $PWD
!ls

/content/pwcnet
datahandler	losses.py		   modules.py	       test.py
flow_color.pdf	model_1000epochs	   README.md	       train.py
flow_utils.py	model_100epochs_ft_Chairs  requirements.txt    utils.py
__init__.py	model_250epochs_ft_Final   sample
LICENSE		model.py		   test_continuous.py


####check if there is GPU available, if available then print its name and device number

In [ ]:
# edit *.py files to add line for GPU to use.
import tensorflow as tf
import os
device_name = tf.test.gpu_device_name()
print(device_name)
if device_name != '/device:GPU:0':
  os.system('sed -i "s/os.environ\\[\'CUDA_VISIBLE_DEVICES\'\\] = input/os.environ[\'CUDA_VISIBLE_DEVICES\'] = \'-1\' # input/g" /content/pwcnet/*.py')
  raise SystemError('GPU device not found')
else:
  os.system('sed -i "s/os.environ\\[\'CUDA_VISIBLE_DEVICES\'\\] = input/os.environ[\'CUDA_VISIBLE_DEVICES\'] = \'0\' # input/g" /content/pwcnet/*.py')
print('Found GPU at: {}'.format(device_name))

/device:GPU:0
Found GPU at: /device:GPU:0


#### command for adding tensorflow 1 compatibility as current google colab VM has tensorflow 2 so some commands won't work

ayushi, you solved this error of **"can't read ./*.py: No such file or directory"**. Can you resolve it again? It seems the issue came again or the new command was not stored correctly

In [ ]:
!sed -i 's/^\s*import tensorflow as.*/import tensorflow.compat.v1 as tf\ntf.disable_v2_behavior()/g' ./*.py

sed: can't read ./*.py: No such file or directory


In [ ]:
#!sed -i 's/from\smodules/from pwcnet.modules/g' pwcnet/*.py | grep modules

####command for training PWCNET on KITTI dataset. 
```please note that the crop shape should be the multiple of 64```

In [ ]:

#[384, 448]

# Added --crop_shape 320 1216 - Avinash
!python train.py --dataset KITTI --dataset_dir /content/Kitti2015 --crop_shape 320 1216

In [ ]:
#!wget https://drive.google.com/drive/folders/1q3pKFc3hThZ4mR4mHV1_sxqX09GeoF7W?usp=sharing
!unzip "/content/1q3pKFc3hThZ4mR4mHV1_sxqX09GeoF7W?usp=sharing" -d "/content/kitti_odometry"

Archive:  /content/1q3pKFc3hThZ4mR4mHV1_sxqX09GeoF7W?usp=sharing
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/1q3pKFc3hThZ4mR4mHV1_sxqX09GeoF7W?usp=sharing or
        /content/1q3pKFc3hThZ4mR4mHV1_sxqX09GeoF7W?usp=sharing.zip, and cannot find /content/1q3pKFc3hThZ4mR4mHV1_sxqX09GeoF7W?usp=sharing.ZIP, period.

No zipfiles found.


####Mounting Google Drive on the VM(current session)
```Please note that if your session expired or gets closed then the drive will be automatically unmounted from the VM```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


####remove a folder

In [ ]:
!rm -rf "/content/pwcnet/"

####Unzip command

In [ ]:
!unzip "/content/drive/MyDrive/VO/pwcnetzip/pwcnet.zip" -d "/content/pwcnet"

####Repetitively moving a directory from one place to another

In [ ]:
#!mkdir Pwcnet
!mv -R "/content/pwcnet/content/pwcnet/*" "/content/Pwcnet"


####Testing the trained PWCNET(continuously iterating over the set of images) by taking stored model weights(three files are stored by PWCNET after 1 successful epoch training

In [ ]:
# %cd pwcnet
# figures will be in test_figure folder
!python test_continuous.py -i /content/Kitti2015/testing/image_2/* -r /content/drive/MyDrive/VO/colab_pwcnet/model/model_100.ckpt

python3: can't open file 'test_continuous.py': [Errno 2] No such file or directory


####Testing the trained PWCNET(takes only two images) by taking stored model weights(three files are stored by PWCNET after 1 successful epoch training

In [ ]:

!python /content/Pwcnet/pwcnet/test.py --input_images /content/Sintel/training/final/alley_1/frame_0004.png /content/Sintel/training/final/alley_1/frame_0005.png --resume /content/drive/MyDrive/VO/colab_pwcnet/model/model_100.ckpt

####Using CV2 library to get shape of an image

In [ ]:
import cv2
img = cv2.imread("/content/test_figure/test_alley_1_frame_0004.png")
print(img.shape)
print(img)
#(438, 1170, 3)

####Zip a folder

In [ ]:
!zip -r "/content/drive/MyDrive/VO/pwcnetzip/pwcnet"  "/content/pwcnet"

#ResNet

####Import necessary modules

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

####Get pretrained ResNet50 model with weights trained using ImageNet dataset without including the top(classification) layer

In [ ]:
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(438, 1170, 3))


94773248/94765736 [==============================] - 1s 0us/step


####Print the ResNet50 Architecture

In [ ]:
base_model.summary()


####Define the top NN layers for R and T matrices(total 6 final values)

In [ ]:


Rot = tf.keras.models.Sequential()
Rot.add(tf.keras.Input(shape=(2072,))) #change the input shape based on the flatten layer 
Rot.add(tf.keras.layers.Dense(128, activation='relu'))
Rot.add(tf.keras.layers.Dense(32, activation='relu'))
Rot.add(tf.keras.layers.Dense(3))

Trans = tf.keras.models.Sequential()
Trans.add(tf.keras.Input(shape=(2072,)))
Trans.add(tf.keras.layers.Dense(128, activation='relu'))
Trans.add(tf.keras.layers.Dense(32, activation='relu'))
Trans.add(tf.keras.layers.Dense(3))


####Add all these above defined layers to the pretrained model and construct the architecture again. Concatenate both 3,3 values in the final layer and give the output

In [ ]:
#(320,1216,3)
inputs = tf.keras.Input(shape=(438, 1170, 3))
x = base_model(inputs)
x = tf.keras.layers.Conv2D(4, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
flatten_layer = tf.keras.layers.Flatten()(x)
rot_outputs = Rot(flatten_layer)
trans_outputs = Trans(flatten_layer)
concatted = tf.keras.layers.Concatenate()([rot_outputs, trans_outputs])


####Define the final model

In [ ]:
ResNet_model = tf.keras.Model(inputs, concatted)

####Recheck the model(by printing summary)

In [ ]:
ResNet_model.summary()

In [ ]:
#train_generator = train_datagen.flow_from_directory(
     # train_dir,
     #  target_size=(48,48),
     #   batch_size=64,
     #    color_mode="gray_framescale",
     #    class_mode='categorical')
from keras.optimizers import Adam
ResNet_model.compile(loss=tf.keras.losses.KLDivergence(),optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
history=ResNet_model.fit(x_train,y_train,batch_size=32,epochs=10,verbose=1,validation_data=(x_test,y_test),callbacks=[check_point])

#ResNet_model.save_weights('model.h5')
#model_info = ResNet_model.fit_generator(
#        train_generator,
#        steps_per_epoch=28709 // 64,
#        epochs=50,
#        validation_data=validation_generator,
#        validation_steps=7178 // 64)

for image_to_rotate in os.listdir(path):
        #do training
#change this
prediction = emotion_model.predict(cropped_img)

####Editing the test file(which is under PWCNET's repo) code and making it a module so that it can be integrated with ResNet in the final model 

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
from pwctest import Tester

#args = {"input_images" = ['/content/Sintel/training/final/alley_1/frame_0004.png', '/content/Sintel/training/final/alley_1/frame_0005.png'], "resume" = '/gdrive/MyDrive/VO/colab_pwcnet/model/model_100.ckpt',"time" = False}
args = {}
args ["input_images"] = ['/content/Sintel/training/final/alley_1/frame_0004.png', '/content/Sintel/training/final/alley_1/frame_0005.png']
args["resume"] = '/content/drive/MyDrive/VO/colab_pwcnet/model/model_100.ckpt'
args["time"] = False
Tester(args)

In [ ]:
!python test_continuous.py -i /content/drive/MyDrive/VO/kitti_odo/Kitti_odometry_Dataset/00/image_0/* -r /content/drive/MyDrive/VO/colab_pwcnet/model/model_100.ckpt

####Path to the optical flow images which were generated using PWCNET and Kitti's 00 image set

In [ ]:
/content/drive/MyDrive/VO/pwcnetzip/pwcnet_unzipped/content/pwcnet/test_figure/image_0/000000.png

#TartanVO

####code meant to work on tartanVO github repository

In [ ]:
!python vo_trajectory_from_folder.py  --model-name tartanvo_1914.pkl --kitti --batch-size 2 --worker-num 0 --test-dir data/KITTI_10/image_left --pose-file data/KITTI_10/pose_left.txt --save-flow

####Meant for TartanVO

In [ ]:
!mkdir models
!wget https://cmu.box.com/shared/static/t1a5u4x6dxohl89104dyrsiz42mvq2sz.pkl -O models/tartanvo_1914.pkl
!git clone https://github.com/castacks/tartanvo.git
!mkdir data
!wget https://cmu.box.com/shared/static/nw3bi7x5vng2xy296ndxt19uozpk64jq.zip -O data/KITTI_10.zip
!unzip -q data/KITTI_10.zip -d data/KITTI_10/
!mkdir data
!wget https://cmu.box.com/shared/static/1ctocidptdv1xev6pjxdj0b782mrstps.zip -O data/EuRoC_V102.zip
!unzip -q data/EuRoC_V102.zip -d data/EuRoC_V102/